Project : 멋진 챗봇 만들기
===

트랜스포머 구조를 활용해 한국어 챗봇을 직접 구현해 보는 프로젝트를 진행한다. 또한 번역기 성능 측정법을 챗봇에도 적용해 봅니다. 

[목차]
1. 데이터 다운로드
2. 데이터 정제
3. 데이터 토큰화
4. Augmentation
5. 데이터 벡터화
6. 훈련하기
7. 성능 측정하기

In [1]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

print(tf.__version__)

2.4.1


# 1. 데이터 다운로드

아래 링크에서 ChatbotData.csv 를 다운로드해 챗봇 훈련 데이터를 확보합니다. pandas 라이브러리로 csv 파일을 읽어봅니다. 읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장합니다. 
- [songys/Chatbot_data](https://github.com/songys/Chatbot_data)

In [2]:
!ls ~/aiffel/GoingDeeper/nlp12/data

'ChatbotData .csv'   ko.bin


In [3]:
data = pd.read_csv('~/aiffel/GoingDeeper/nlp12/data/ChatbotData .csv')

data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [5]:
data.drop(("label"), axis="columns", inplace=True)

data.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


# 2. 데이터 정제

## 중복 제거
먼저, 중복 데이터를 확인하고 제거해 줍니다. 

In [6]:
data[data.duplicated()]

,Q,A
5527,결혼이나 하지 왜 자꾸 나한테 화 내냐구!,힘들겠네요.
5537,고백하고 후회하면 어떡하지,후회는 후회를 낳을뿐이에요. 용기 내세요.
5542,공부는 내 체질이 아닌 것 같아,확신이 없나봐요.
5704,기숙사 괜찮을까,혼자 사는 것보다 불편하겠죠.
5774,나는 좋은데 ….,현실의 벽에 부딪혔나봐요.
...,...,...
8764,환승 가능?,환승은 30분 안에
8780,회사 사람들이 아직도 불편해,회사에는 동료가 있을 뿐이에요.
8782,회사에는 왜 친구 같은 사람이 없을까,회사는 친구 사귀는 곳이 아니에요.
8789,후련하달까,후련하니 다행이에요.


In [7]:
data.drop_duplicates(inplace=True)
print(len(data))

11750


In [13]:
questions, answers = list(data['Q']), list(data['A'])

print(len(questions), len(answers))

11750 11750


## 전처리 함수 

이제, 아래 조건을 만족하는 preprocess_sentence() 함수를 구현합니다.

1. 영문자의 경우, 모두 소문자로 변환합니다.
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.
   
문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않습니다. 

In [9]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^0-9가-힣a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    
    return sentence

# 3. 데이터 토큰화

토큰화에는 KoNLPy의 mecab 클래스를 사용합니다.
- [Mecab Class](https://konlpy.org/ko/latest/api/konlpy.tag/#mecab-class)

## 토큰화 함수

아래 조건을 만족하는 build_corpus() 함수를 구현합니다. 

1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다.
3. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달합니다.
4. 중복되는 문장은 데이터에서 제외합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의합니다. 

구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.

In [21]:
def build_corpus(src, tgt, tokenizer):
    
    src_corpus = []
    tgt_corpus = []
    
    
    for i in range(len(src)):
        src_sen = preprocess_sentence(src[i])
        tgt_sen = preprocess_sentence(tgt[i])

        src_tokens = tokenizer.morphs(src_sen)
        tgt_tokens = tokenizer.morphs(tgt_sen)
        
        if src_tokens in src_corpus and tgt_tokens in tgt_corpus: continue
        
        src_corpus.append(src_tokens)
        tgt_corpus.append(tgt_tokens)
        
    return src_corpus, tgt_corpus

In [22]:
from konlpy.tag import Mecab
tokenizer = Mecab()

que_corpus, ans_corpus = build_corpus(questions, answers, tokenizer)
print(len(que_corpus), len(ans_corpus))

11719 11719


In [23]:
print(que_corpus[:5])
print(ans_corpus[:5])

[['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'], ['ppl', '심하', '네']]
[['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['눈살', '이', '찌푸려', '지', '죠', '.']]


## 토큰 개수 제한

토큰 개수 분포를 확인하여 토큰의 개수가 일정 길이 이상은 데이터에서 제외합니다.

In [24]:
total_data = list(que_corpus) + list(ans_corpus)

num_tokens = [len(tokens) for tokens in total_data]
num_tokens = np.array(num_tokens)

# 토큰개수의 평균값, 최대값, 표준편차 계산
print('토큰개수 평균 : ', np.mean(num_tokens))
print('토큰개수 최대 : ', np.max(num_tokens))
print('토큰개수 표준편차 : ', np.std(num_tokens))

토큰개수 평균 :  7.7065449270415565
토큰개수 최대 :  40
토큰개수 표준편차 :  3.6117207348364158


In [25]:
# 최대 길이를 (평균 + 2.3*표준편차)로 지정
max_tokens = np.mean(num_tokens) + 2.3 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 데이터의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

pad_sequences maxlen :  16
전체 데이터의 0.9756805188155986%가 maxlen 설정값 이내에 포함됩니다. 


토큰의 개수가 16개 이상인 데이터는 제외시켜 줍니다. 

In [34]:
que_corpus2 = []
ans_corpus2 = []

for i in range(len(que_corpus)):
    if len(que_corpus[i]) < maxlen or len(ans_corpus[i]) < maxlen: 
        que_corpus2.append(que_corpus[i])
        ans_corpus2.append(ans_corpus[i])
        
print(len(que_corpus2), len(ans_corpus2))

11058 11058


In [ ]:
# 데이터의 1%는 test 데이터로 남겨둡니다. 

# 4. Augmentation

우리에게 주어진 데이터는 1만 개가량으로 적은 편에 속합니다. 이럴 때에 사용할 수 있는 Lexical Substitution을 실제로 적용해 봅니다. 
    
아래 링크를 참고하여 한국어로 사전 훈련된 Embedding 모델을 다운로드합니다. Korean (w) 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 Korean (w)를 찾아 다운로드하고, ko.bin 파일을 얻습니다. 
- [Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)
    
다운로드한 모델을 활용해 데이터를 Augmentation 합니다.
    
1) Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록 합니다.     
2) 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 합니다.  

위의 과정으로 전체 데이터가 원래의 3배가량으로 늘어나도록 합니다.

In [27]:
import gensim

w2v_path = os.getenv('HOME')+'/aiffel/GoingDeeper/nlp12/data/ko.bin'
w2v = gensim.models.Word2Vec.load(w2v_path)

In [50]:
# Lexical Substitution 구현하기
def lexical_sub(sentence, word2vec):
    import random
    
    res = []

    try:
        _from = random.choice(sentence)
        _to = word2vec.most_similar(_from)[0][0]
        
    except:   # 단어장에 없는 단어
        return None

    for tok in sentence:
        if tok is _from: res.append(_to)
        else: res.append(tok)

    return res

In [51]:
from tqdm import tqdm

# 1) que_corpus Augmentation
new_que = []

for idx in tqdm(range(len(que_corpus2))):
    old_src = que_corpus2[idx]

    new_src = lexical_sub(old_src, w2v)

    if new_src is not None: new_que.append(new_src)
    else: new_que.append(old_src)

print(new_que[:10])
print(len(new_que))

  0%|          | 0/11058 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':
100%|██████████| 11058/11058 [00:19<00:00, 575.50it/s]

[['12', '시', '땡', '캐치'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '김', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '겠', '다'], ['ppl', '심하', '네'], ['sd', '카드', '망가졌', '어'], ['sd', '카드', '못가', '돼'], ['sns', '들어맞', '팔', '왜', '안', '하', '지'], ['sns', '시간', '낭비', '인', '거', '아', '으며', '매일', '하', '는', '중'], ['sns', '시간', '낭비', '인데', '자꾸', '살펴보', '게', '됨']]


In [52]:
len(new_que)

11058

In [55]:
# Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록 
que_corpus3 = []
ans_corpus3 = []

for i in range(len(new_que)): 
    if new_que[i] not in que_corpus2: # 중복 제거 
        que_corpus3.append(new_que[i])
        ans_corpus3.append(ans_corpus2[i])
        
print(len(que_corpus3), len(ans_corpus3))
print(que_corpus3[:5])
print(ans_corpus3[:5])

9344 9344
[['12', '시', '땡', '캐치'], ['3', '김', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '겠', '다'], ['sd', '카드', '못가', '돼'], ['sns', '들어맞', '팔', '왜', '안', '하', '지']]
[['하루', '가', '또', '가', '네요', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['다시', '새로', '사', '는', '게', '마음', '편해요', '.'], ['잘', '모르', '고', '있', '을', '수', '도', '있', '어요', '.']]


In [56]:
# 2) ans_corpus Augmentation
new_ans = []

for idx in tqdm(range(len(ans_corpus2))):
    old_src = ans_corpus2[idx]

    new_src = lexical_sub(old_src, w2v)

    if new_src is not None: new_ans.append(new_src)
    else: new_ans.append(old_src)

print(new_ans[:10])
print(len(new_ans))

  0%|          | 0/11058 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':
100%|██████████| 11058/11058 [00:19<00:00, 555.49it/s]

[['하루', '가', '각기', '가', '네요', '.'], ['무릎', '해', '드립니다', '.'], ['여행', '은데', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '괜찮', '죠', '.'], ['눈살', '그러', '찌푸려', '지', '죠', '.'], ['다시', '새로', '타', '는', '게', '마음', '편해요', '.'], ['다시', '새로이', '사', '는', '게', '마음', '편해요', '.'], ['많이', '모르', '고', '있', '을', '수', '도', '있', '어요', '.'], ['시간', '을', '정하', '고', '해', '보', 'ㅂ시오', '.'], ['시간', '을', '정해지', '고', '해', '보', '세요', '.']]


In [57]:
len(new_ans)

11058

In [58]:
# 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 
que_corpus4 = []
ans_corpus4 = []

for i in range(len(new_ans)): 
    if new_ans[i] not in ans_corpus2: # 중복 제거 
        que_corpus4.append(que_corpus2[i])
        ans_corpus4.append(new_ans[i])
        
print(len(que_corpus4), len(ans_corpus4))
print(que_corpus4[:5])
print(ans_corpus4[:5])

9697 9697
[['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'], ['ppl', '심하', '네']]
[['하루', '가', '각기', '가', '네요', '.'], ['무릎', '해', '드립니다', '.'], ['여행', '은데', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '괜찮', '죠', '.'], ['눈살', '그러', '찌푸려', '지', '죠', '.']]


In [59]:
que_corpus = que_corpus2 + que_corpus3 + que_corpus4
ans_corpus = ans_corpus2 + ans_corpus3 + ans_corpus4

print(len(que_corpus), len(ans_corpus))

30099 30099


총 3만개의 데이터로 전체 데이터가 원래의 3배가량으로 늘어난 것을 볼 수 있습니다. 

# 5. 데이터 벡터화

타겟 데이터인 ans_corpus 에 \<start> 토큰과 \<end> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다.    
챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것이겠죠. 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있습니다.
    
1) 타겟 데이터 전체에 \<start> 토큰과 \<end> 토큰을 추가해 줍니다.         
2) 특수 토큰을 더함으로써 ans_corpus 또한 완성이 되었으니, que_corpus 와 결합하여 전체 데이터에 대한 단어 사전을 구축하고 벡터화하여 enc_train 과 dec_train 을 얻습니다. 

In [60]:
for i in range(len(ans_corpus)):
    ans_corpus[i] = ["<start>"] + ans_corpus[i] + ["<end>"]
    
print(ans_corpus[:5])

[['<start>', '하루', '가', '또', '가', '네요', '.', '<end>'], ['<start>', '위로', '해', '드립니다', '.', '<end>'], ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'], ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'], ['<start>', '눈살', '이', '찌푸려', '지', '죠', '.', '<end>']]
